In [1]:
import sklearn
import pandas as pd

import logging

import torch
cuda_available = torch.cuda.is_available()

from simpletransformers.ner import NERModel, NERArgs

d:\Drive\KULEEAH\Semester 8\TA Farihin\env\lib\site-packages\torchaudio\backend\utils.py:67: UserWarning: No audio backend is available.
  warnings.warn('No audio backend is available.')


In [2]:
train_df=pd.read_csv("..\..\..\Dataset\\bert\\train.csv")
eval_df=pd.read_csv("..\..\..\Dataset\\bert\\val.csv")


In [3]:
train_df=train_df
train_df

,sentence_id,words,labels
0,1382601382042103808,Hidup,O
1,1382601382042103808,sesedih,O
2,1382601382042103808,dan,O
3,1382601382042103808,secaper,O
4,1382601382042103808,apa,O
...,...,...,...
108840,1383795163412058114,Gubernur,O
108841,1383795163412058114,Khofifah,B-PER
108842,1383795163412058114,fokus,O
108843,1383795163412058114,untuk,O


In [4]:
eval_df

,sentence_id,words,labels
0,1385573262067847169,Nadiem,B-PER
1,1385573262067847169,Makarim,I-PER
2,1385573262067847169,dg,O
3,1385573262067847169,Presiden,O
4,1385573262067847169,ke-5,O
...,...,...,...
27206,1382913410749190149,aku,O
27207,1383795163412058114,ekonomi,O
27208,1383795163412058114,",",O
27209,1383795163412058114,infrastruktur,O


In [5]:
label_list = [
    "O", 
    "B-PER", "I-PER",
    "B-ORG", "I-ORG",
    "B-LOC", "I-LOC",
    "B-PROD", "I-PROD",
    "B-WA", "I-WA",
    "B-EV", "I-EV",
]

In [6]:
# Create a NERModel
model_args=NERArgs()
# model_args.train_batch_size=64
# model_args.overwrite_output_dir=True
model_args = {
    "output_dir": "outputs/",
    "cache_dir": "cache_dir/",

    "fp16": True,
    "fp16_opt_level": "O1",
    "max_seq_length": 128,
    "train_batch_size": 8,
    "gradient_accumulation_steps": 1,
    "eval_batch_size": 8,
    "num_train_epochs": 1,
    # "num_train_epochs": 5,
    "weight_decay": 0,
    "learning_rate": 4e-5,
    # "learning_rate": 0.00006828,

    "adam_epsilon": 1e-8,
    "warmup_ratio": 0.06,
    "warmup_steps": 0,
    "max_grad_norm": 1.0,

    "logging_steps": 50,
    "save_steps": 2000,

    "overwrite_output_dir": True,
    "reprocess_input_data": False,
    "evaluate_during_training": True,
}


In [7]:
# model = NERModel('bert', 
#                 'outputs',
#                 args=model_args,
#                 labels=label_list,
#                 use_cuda=cuda_available,
#                 )
# # result, model_outputs, wrong_preds = model.eval_model(eval_df)
# # result

In [8]:
model = NERModel('bert', 
                'bert-base-multilingual-cased',
                args=model_args,
                labels=label_list,
                use_cuda=cuda_available,
                )

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForTokenClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at 

In [17]:
torch.cuda.empty_cache()
torch.cuda.memory_summary(device=None, abbreviated=False)

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 4            |        cudaMalloc retries: 5         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |    3047 MB |    3057 MB |   16017 MB |   12969 MB |\n|       from large pool |    3039 MB |    3047 MB |   15935 MB |   12895 MB |\n|       from small pool |       8 MB |      21 MB |      82 MB |      73 MB |\n|---------------------------------------------------------------------------|\n| Active memory         |    3047 MB |    3057 MB |   16017 MB |   12969 MB |\n|       from large pool |    3039 MB |    3047 MB |

In [18]:
    model.train_model(train_df,  eval_data=eval_df)

Epoch 1 of 1:   0%|          | 0/1 [00:01<?, ?it/s]


RuntimeError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 4.00 GiB total capacity; 2.98 GiB already allocated; 0 bytes free; 3.00 GiB reserved in total by PyTorch)

In [ ]:
result, model_outputs, wrong_preds = model.eval_model(eval_df)

In [ ]:
result

In [ ]:
predictions, raw_outputs = model.predict(
    # ["@PolsekAlian PolriBerikan inovasi-inovasi Polri yang memudahkan masyarakat, kegiatan pemolisian masyarakat dan seluruh tindakan Kepolisian dalam menghadapi gangguan Kamtibmas, dan lain-lain - Jenderal Polisi Drs. Listyo Sigit Prabowo, https://t.co/f8GZ6VPqPD. - Kapolri #PolriTVRadioPresisi"],
    ["Safari Ramadhan di Malang, Gubernur Khofifah ingatkan pentingnya Rembuk Nyekrup birokrat"], 
    ["@OPPOIndonesia ✍Butuh memori besar 128GB di OPPO A54 buat simpan file-file tugas kuliah dan testimoni online shop -ku ��Semoga oppo A54 bisa menunjang penghasilan keluargaku �� @OPPOIndonesia @nisa_rkt @Bebheey @kus_ica @Erna19_ @Saftriyuni #OPPOA54 https://t.co/bMYIiRb4QF �� 2.921"]
        )

In [ ]:
predictions